In [50]:
!pip install backtrader
!pip install yfinance
!pip install matplotlib
!pip install pandas_ta

In [51]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats
import backtrader.indicators
import datetime
import yfinance as yf
import matplotlib as plt
import pandas_ta as ta

In [129]:
class MAstrategy(bt.Strategy):
    def __init__(self):
        self.ma20 = bt.indicators.ExponentialMovingAverage(self.data.close, period=20)
        self.ma25 = bt.indicators.ExponentialMovingAverage(self.data.close, period=25)
        self.ma30 = bt.indicators.ExponentialMovingAverage(self.data.close, period=30)
        self.ma35 = bt.indicators.ExponentialMovingAverage(self.data.close, period=35)
        self.ma40 = bt.indicators.ExponentialMovingAverage(self.data.close, period=40)
        self.ma45 = bt.indicators.ExponentialMovingAverage(self.data.close, period=45)
        self.ma50 = bt.indicators.ExponentialMovingAverage(self.data.close, period=50)
        self.ma55 = bt.indicators.ExponentialMovingAverage(self.data.close, period=55)
        self.volume_avg = bt.indicators.SimpleMovingAverage(self.data.volume, period=10)
        self.rsi = bt.indicators.RelativeStrengthIndex()
        self.order = None
        self.buy_price = None
        self.stop_price = None

    def next(self):
        if self.order:
            return
        if not self.position:
            ma_conditions_asc = [
                self.ma20[0] > self.ma25[0],
                self.ma25[0] > self.ma30[0],
                self.ma30[0] > self.ma35[0],
                self.ma35[0] > self.ma40[0],
                self.ma40[0] > self.ma45[0],
                self.ma45[0] > self.ma50[0],
                self.ma50[0] > self.ma55[0] and self.data.close[0] > self.ma20[0]and self.data.open[0] < self.ma55[0]
            ]

            ma_conditions_desc = [
                self.ma20[0] < self.ma25[0],
                self.ma25[0] < self.ma30[0],
                self.ma30[0] < self.ma35[0],
                self.ma35[0] < self.ma40[0],
                self.ma40[0] < self.ma45[0],
                self.ma45[0] < self.ma50[0],
                self.ma50[0] < self.ma55[0] and
                self.data.open[0] < self.ma20[0]and self.data.close[0] > self.ma55[0]  # Corrected line
            ]

            if (
                (all(ma_conditions_asc) or all(ma_conditions_desc)) and

                (self.data.volume[0] > self.volume_avg[0])
            ):
                self.log('Buy Create, %.2f' % self.data.close[0])
                self.order = self.buy(size=100)
                self.buy_price = self.data.close[0]
                self.stop_price = self.buy_price * 0.95
                self.log('Stop Loss set at %.2f' % self.stop_price)

        else:
            if self.data.high[0] >= self.buy_price * 1.05:
                self.log('Sell Create, %.2f' % self.data.close[0])
                self.order = self.sell(size=100)
            elif self.data.low[0] <= self.stop_price:
                self.log('Stop Loss triggered, %.2f' % self.data.close[0])
                self.order = self.sell(size=100)

    def log(self, txt):
        dt = self.data.datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    "Executed BUY (Price: %.2f, Value: %.2f, Commission %.2f)" %
                    (order.executed.price, order.executed.value, order.executed.comm))
                self.buy_price = order.executed.price
                self.stop_price = self.buy_price * 0.95
            elif order.issell():
                self.log(
                    "Executed SELL (Price: %.2f, Value: %.2f, Commission %.2f)" %
                    (order.executed.price, order.executed.value, order.executed.comm))

        self.order = None



In [138]:
cerebro = bt.Cerebro() # Initialise the bot


feed = bt.feeds.PandasData(dataname=yf.download('JINDALSTEL.NS', '2000-01-01', '2024-04-30')) # To set the data feed
cerebro.adddata(feed) # To feed the data into Cerebro engine

cerebro.addstrategy(MAstrategy) # To add the strategy
cerebro.broker.setcash(20000.0) # To set initial portfolio value
cerebro.broker.setcommission(commission=.0002) # To set broker commission

#cerebro.addsizer(bt.sizers.PercentSizer, percents=90) # To allocate percentage of portfolio to execute an order

[*********************100%***********************]  1 of 1 completed


In [139]:
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='maxdrawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")

In [140]:
def printDrawDownAnalysis(analyzer): # Function to print the Technical Analysis results in a nice format.

    # Get the results we are interested in
    drawdown = round(analyzer.drawdown, 2)
    moneydown = round(analyzer.moneydown, 2)
    length = analyzer.len
    max_dd = round(analyzer.max.drawdown, 2)
    max_md = round(analyzer.max.moneydown, 2)
    max_len = analyzer.max.len
    h1 = ['Drawdown', 'Moneydown', 'Length']
    h2 = ['Max drawdown','Max moneydown', 'Max len']
    r1 = [drawdown, moneydown,length]
    r2 = [max_dd, max_md, max_len]
    # Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    # Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Drawdown Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))


def printTradeAnalysis(analyzer): # Function to print the Technical Analysis results in a nice format.

    # Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = round((total_won / total_closed) * 100)
    # Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed, total_won, total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    # Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    # Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

In [141]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue()) # Print the initial portfolio value
backtest_result = cerebro.run()
backtest_result_1 = backtest_result[0]
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue()) # Print the final portfolio value
print('Sharpe Ratio:', backtest_result_1.analyzers.mysharpe.get_analysis())
printDrawDownAnalysis(backtest_result_1.analyzers.maxdrawdown.get_analysis()) # To get detailed Drawdown and Trade analysis
printTradeAnalysis(backtest_result_1.analyzers.ta.get_analysis())

Starting Portfolio Value: 20000.00
2000-12-07, Buy Create, 2.68
2000-12-07, Stop Loss set at 2.54
2000-12-08, Executed BUY (Price: 2.63, Value: 262.67, Commission 0.05)
2000-12-08, Sell Create, 2.89
2000-12-11, Executed SELL (Price: 2.96, Value: 262.67, Commission 0.06)
2001-01-11, Buy Create, 3.05
2001-01-11, Stop Loss set at 2.90
2001-01-12, Executed BUY (Price: 3.12, Value: 312.50, Commission 0.06)
2001-01-12, Sell Create, 3.30
2001-01-15, Executed SELL (Price: 3.53, Value: 312.50, Commission 0.07)
2001-04-30, Buy Create, 3.96
2001-04-30, Stop Loss set at 3.77
2001-05-01, Executed BUY (Price: 3.96, Value: 396.33, Commission 0.08)
2001-05-02, Stop Loss triggered, 3.65
2001-05-03, Executed SELL (Price: 3.67, Value: 396.33, Commission 0.07)
2004-07-12, Buy Create, 15.70
2004-07-12, Stop Loss set at 14.92
2004-07-13, Executed BUY (Price: 16.06, Value: 1606.33, Commission 0.32)
2004-07-16, Sell Create, 17.68
2004-07-19, Executed SELL (Price: 17.91, Value: 1606.33, Commission 0.36)
2005-0